# OpenQasm 2.0

[OpenQasm](https://github.com/Qiskit/openqasm/tree/OpenQASM2.x) is a programming language aimed to become the central pilar to program quantum computers in the future. However, this programming language it's still under development so beware of the bugs you may encounter. When you are coding in OpenQasm you can almost feel like you are programming in the C language. It's an [imperative language](https://en.wikipedia.org/wiki/Imperative_programming) that can represent all of our circuits in a lower level of abstraction. Some general characteristics of it's grammar are: 
- sentences use semicolons
- it's case sensitive
- white spaces are ignored
- It supports single line comments (// A comment line)
- Types must be declared (qreg gamma[1];)

Now that we now some quick facts let's dig deeper into its grammar.

## Variables:

You can declare two types of variables: quantum quibits and classical registers. Both of them use an array like structure and they are static (once created they never change their size). You can see the summary of the variables in the following table:


| variable | Sintaxis |
|--------------|----------------|
| qubits           | qreg array[20];    |
| classical register           | creg bit_array[20];    |



## Gates:

You can declare gates in the following way:

                                       gate(*args) Targetqubit; 

or

                                 gate(*args) Targetqubit, Controllerqubit;

Check this table for more examples with the most used gates:

| gate | Sintaxis                       |
|--------------|------------------------|
| U3           | u3(pi,0,pi) q[0];    |
| X            | x q[0];                |
| H            | h q[0];                |
| CNOT         | cx q[0],q[1];       |


## Measurements:

You can declare a measurement with the following sintax:

                                        measure q[0] -> c[0];
                            
After that you'll have the result of the measurement of the qubit 0 in the classical register 0.

## Runing a .qasm file

### OQI
Now, to run a .qasm file we have two options. We can run it with a pip package that I created. That is, execute it with [OQI](https://github.com/mentesniker/OpenQasmInterpreter). It's pretty simple, once is installed you just have to run the following command:

                                          oqi nameFile.qasm

After executing this command you'll have the result from your circuit. For example:

                                      The results are: {'0': 100}

### Qiskit
You can also translate a .qasm file into a quantum circuit. This is done with the function from_qasm_file(). From that point forward you can visualize or run it with the different methods qiskit has. For example if we had a "circuit.qasm" file, then the result would be something like the following:

In [5]:
from qiskit import *
newQc = QuantumCircuit.from_qasm_file("circuit.qasm")
backend = BasicAer.get_backend('qasm_simulator')
job = execute(newQc, backend, shots=100)
result = job.result()
print("\n The results are: " + str(result.get_counts(newQc)) + "\n")


 The results are: {'0': 47, '1': 53}



## Example: Construction of a toffoli gate.

In one of my [previous posts](https://mentesniker.github.io/Quantum-computer-science-tutorials/) I created a toffoli gate from scratch. We'll implement it now using OpenQasm. First of all, let's do a little recap of the toffoli gate. When [we presented the CNOT gate](https://mentesniker.github.io/Quantum-computer-science-tutorials/introduction01.html) we said something like: if the controller qubit is one then when apply an X-Gate on the target qubit. Now the question is, How can we construct a CNOT Gate which have two controller qubits. The answer is the toffoli gate. There are several ways to define a toffoli gate but we'll construct it accordingly to [1]. The main idea is the following:

- If the controller qubits are 0 then nothing is going to happen because no CNOT gate will be applied and the T gates are going to cancell with the T dagger gates.
- If one of the controller qubits are 1, then the T gates are going to cancell the rotation applied by the CNOT Gate.
- If the two of the controller qubits are one then we are going to apply a rotation equivalent to the rotation of a Z-Gate. And after the application of the H-Gates we will have the desired result.

Now, let's build it with OpenQasm:

```
OPENQASM 2.0;
include "qelib1.inc";
qreg target[1];
qreg controllers[2];
creg b[3];
x target[0];
x controllers[0];
h controllers[1];
t controllers[1];
cx controllers[0],controllers[1];
tdg controllers[1];
cx target[0],controllers[1];
t controllers[1];
cx controllers[0],controllers[1];
tdg controllers[1];
h controllers[1];
barrier target[0],controllers[0],controllers[1];
measure controllers[1] -> b[0];
measure controllers[0] -> b[1];
measure target[0] -> b[2];
```

And if we run it with oqi the result will be the following:

```
               ┌───┐                                             ░       ┌─┐
     target_0: ┤ X ├───────────────────■─────────────────────────░───────┤M├
               ├───┤                   │                         ░    ┌─┐└╥┘
controllers_0: ┤ X ├───────■───────────┼─────────■───────────────░────┤M├─╫─
               ├───┤┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌─────┐┌───┐ ░ ┌─┐└╥┘ ║ 
controllers_1: ┤ H ├┤ T ├┤ X ├┤ TDG ├┤ X ├┤ T ├┤ X ├┤ TDG ├┤ H ├─░─┤M├─╫──╫─
               └───┘└───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘ ░ └╥┘ ║  ║ 
          b: 3/═════════════════════════════════════════════════════╩══╩══╩═
                                                                    0  1  2 

 The results are: {'111': 100}
```

## References:

[1] Maslov, Dmitri. (2015). On the advantages of using relative phase Toffolis with an application to multiple control Toffoli optimization. Physical Review A. 93. 10.1103/PhysRevA.93.022311.